## MICCAI 2015 Abdominal dataset

### Overview

The third step: resampling images to 192 192, extract ROI, and re-index labels


In [2]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import sys


sys.path.insert(0, '../../../dataloaders/')
import niftiio as nio



Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
IMG_FOLDER = "./tmp_boundary_removed/"
SEG_FOLDER = IMG_FOLDER
imgs = glob.glob(IMG_FOLDER + "/image_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/label_*.nii.gz")) ]

pids = [pid.split("_")[-1].split(".")[0] for pid in imgs]

In [4]:
imgs

['./tmp_boundary_removed/image_0.nii.gz',
 './tmp_boundary_removed/image_1.nii.gz',
 './tmp_boundary_removed/image_10.nii.gz',
 './tmp_boundary_removed/image_11.nii.gz',
 './tmp_boundary_removed/image_12.nii.gz',
 './tmp_boundary_removed/image_13.nii.gz',
 './tmp_boundary_removed/image_14.nii.gz',
 './tmp_boundary_removed/image_15.nii.gz',
 './tmp_boundary_removed/image_16.nii.gz',
 './tmp_boundary_removed/image_17.nii.gz',
 './tmp_boundary_removed/image_18.nii.gz',
 './tmp_boundary_removed/image_19.nii.gz',
 './tmp_boundary_removed/image_2.nii.gz',
 './tmp_boundary_removed/image_20.nii.gz',
 './tmp_boundary_removed/image_21.nii.gz',
 './tmp_boundary_removed/image_22.nii.gz',
 './tmp_boundary_removed/image_23.nii.gz',
 './tmp_boundary_removed/image_24.nii.gz',
 './tmp_boundary_removed/image_25.nii.gz',
 './tmp_boundary_removed/image_26.nii.gz',
 './tmp_boundary_removed/image_27.nii.gz',
 './tmp_boundary_removed/image_28.nii.gz',
 './tmp_boundary_removed/image_29.nii.gz',
 './tmp_bounda

In [5]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj
        
## Then crop ROI
def get_label_center(label):
    nnz = np.sum(label > 1e-5)
    return np.int32(np.rint(np.sum(np.nonzero(label), axis = 1) * 1.0 / nnz))

def image_crop(ori_vol, crop_size, referece_ctr_idx, padval = 0., only_2d = True):
    """ crop a 3d matrix given the index of the new volume on the original volume
        Args:
            refernce_ctr_idx: the center of the new volume on the original volume (in indices)
            only_2d: only do cropping on first two dimensions
    """
    _expand_cropsize = [x + 1 for x in crop_size] # to deal with boundary case
    if only_2d:
        assert len(crop_size) == 2, "Actual len {}".format(len(crop_size))
        assert len(referece_ctr_idx) == 2, "Actual len {}".format(len(referece_ctr_idx))
        _expand_cropsize.append(ori_vol.shape[-1])
        
    image_patch = np.ones(tuple(_expand_cropsize)) * padval

    half_size = tuple( [int(x * 1.0 / 2) for x in _expand_cropsize] )
    _min_idx = [0,0,0]
    _max_idx = list(ori_vol.shape)

    # bias of actual cropped size to the beginning and the end of this volume
    _bias_start = [0,0,0]
    _bias_end = [0,0,0]

    for dim,hsize in enumerate(half_size):
        if dim == 2 and only_2d:
            break

        _bias_start[dim] = np.min([hsize, referece_ctr_idx[dim]])
        _bias_end[dim] = np.min([hsize, ori_vol.shape[dim] - referece_ctr_idx[dim]])

        _min_idx[dim] = referece_ctr_idx[dim] - _bias_start[dim]
        _max_idx[dim] = referece_ctr_idx[dim] + _bias_end[dim]
        
    if only_2d:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], ... ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], ... ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], : ]
    # then goes back to original volume
    else:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], \
                half_size[2] - _bias_start[2]: half_size[2] +_bias_end[2] ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], \
                referece_ctr_idx[2] - _bias_start[2]: referece_ctr_idx[2] +_bias_end[2] ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], 0: crop_size[2] ]
    return image_patch

   
    
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [6]:
def z_crop(x_in, labels):
    slc_list = []
    for ii in range(x_in.shape[0]):
        for lbv in labels:
            if lbv in np.unique(np.int32(x_in[ii])):
                slc_list.append(ii)
                break
                
    min_slc = np.min(slc_list)
    max_slc = np.max(slc_list)
    return min_slc, max_slc
            
def label_remap(x_in, labelmap = [(1,4), (2,2), (3,3), (6,1) ]):
    '''
    (sabs value, chaos value)
    '''
    x_out = np.zeros(x_in.shape)
    x_in = np.int32(x_in)
    for src, dst in labelmap:
        x_out[x_in == src] = dst
        
    return np.float32(x_out)
        

In [7]:
imgs

['./tmp_boundary_removed/image_0.nii.gz',
 './tmp_boundary_removed/image_1.nii.gz',
 './tmp_boundary_removed/image_10.nii.gz',
 './tmp_boundary_removed/image_11.nii.gz',
 './tmp_boundary_removed/image_12.nii.gz',
 './tmp_boundary_removed/image_13.nii.gz',
 './tmp_boundary_removed/image_14.nii.gz',
 './tmp_boundary_removed/image_15.nii.gz',
 './tmp_boundary_removed/image_16.nii.gz',
 './tmp_boundary_removed/image_17.nii.gz',
 './tmp_boundary_removed/image_18.nii.gz',
 './tmp_boundary_removed/image_19.nii.gz',
 './tmp_boundary_removed/image_2.nii.gz',
 './tmp_boundary_removed/image_20.nii.gz',
 './tmp_boundary_removed/image_21.nii.gz',
 './tmp_boundary_removed/image_22.nii.gz',
 './tmp_boundary_removed/image_23.nii.gz',
 './tmp_boundary_removed/image_24.nii.gz',
 './tmp_boundary_removed/image_25.nii.gz',
 './tmp_boundary_removed/image_26.nii.gz',
 './tmp_boundary_removed/image_27.nii.gz',
 './tmp_boundary_removed/image_28.nii.gz',
 './tmp_boundary_removed/image_29.nii.gz',
 './tmp_bounda

In [8]:
segs

['./tmp_boundary_removed/label_0.nii.gz',
 './tmp_boundary_removed/label_1.nii.gz',
 './tmp_boundary_removed/label_10.nii.gz',
 './tmp_boundary_removed/label_11.nii.gz',
 './tmp_boundary_removed/label_12.nii.gz',
 './tmp_boundary_removed/label_13.nii.gz',
 './tmp_boundary_removed/label_14.nii.gz',
 './tmp_boundary_removed/label_15.nii.gz',
 './tmp_boundary_removed/label_16.nii.gz',
 './tmp_boundary_removed/label_17.nii.gz',
 './tmp_boundary_removed/label_18.nii.gz',
 './tmp_boundary_removed/label_19.nii.gz',
 './tmp_boundary_removed/label_2.nii.gz',
 './tmp_boundary_removed/label_20.nii.gz',
 './tmp_boundary_removed/label_21.nii.gz',
 './tmp_boundary_removed/label_22.nii.gz',
 './tmp_boundary_removed/label_23.nii.gz',
 './tmp_boundary_removed/label_24.nii.gz',
 './tmp_boundary_removed/label_25.nii.gz',
 './tmp_boundary_removed/label_26.nii.gz',
 './tmp_boundary_removed/label_27.nii.gz',
 './tmp_boundary_removed/label_28.nii.gz',
 './tmp_boundary_removed/label_29.nii.gz',
 './tmp_bounda

In [9]:
import copy
#OUT_FOLDER = "./sabs192c/"
OUT_FOLDER = "./processed"
scan_dir = OUT_FOLDER
os.makedirs(scan_dir, exist_ok = True)
BD_BIAS = 0# cut irrelavent empty boundary to make roi stands out

SPA_FAC = 256 / 192 # spacing factor

for img_fid, seg_fid, pid in zip(imgs, segs, pids):

    lb_n = nio.read_nii_bysitk(seg_fid)

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    ## image
    array = sitk.GetArrayFromImage(img_obj)
    # cropping
    array = array[:, : 256, : 256]
    cropped_img_o = sitk.GetImageFromArray(array)
    cropped_img_o = copy_spacing_ori(img_obj, cropped_img_o)

    # resampling
    img_spa_ori = img_obj.GetSpacing()
    res_img_o = resample_by_res(cropped_img_o, [img_spa_ori[0] * SPA_FAC, img_spa_ori[1] * SPA_FAC, img_spa_ori[-1]], interpolator = sitk.sitkLinear,
                                    logging = True)

    ## label
    lb_arr = sitk.GetArrayFromImage(seg_obj)
    
    # cropping
    lb_arr = lb_arr[:,: 256, : 256]
    cropped_lb_o = sitk.GetImageFromArray(lb_arr)
    cropped_lb_o = copy_spacing_ori(seg_obj, cropped_lb_o)

    lb_spa_ori = seg_obj.GetSpacing()

    # resampling
    res_lb_o = resample_lb_by_res(cropped_lb_o,  [lb_spa_ori[0] * SPA_FAC, lb_spa_ori[1] * SPA_FAC, lb_spa_ori[-1] ], interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)

    
    # in case of resampling bug
    res_img_a = sitk.GetArrayFromImage(res_img_o)[: , : 192, : 192]
    res_lb_a = sitk.GetArrayFromImage(res_lb_o)[: , : 192, : 192]
    
    
    ## ROI
    min_slc, max_slc = z_crop(res_lb_a, labels = [1,2,3,6])
    res_img_a  = res_img_a[min_slc: max_slc, ...]
    res_lb_a = res_lb_a[min_slc: max_slc, ...]
    res_lb_a = label_remap(res_lb_a)
    
    out_img_o = sitk.GetImageFromArray(res_img_a)
    out_img_o = copy_spacing_ori(img_obj, out_img_o)

    out_lb_o = sitk.GetImageFromArray(res_lb_a)
    out_lb_o = copy_spacing_ori(img_obj, out_lb_o)
   
    
    out_img_fid = os.path.join( scan_dir, f'image_{pid}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{pid}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(out_img_o, out_img_fid, True) 
    sitk.WriteImage(out_lb_o, out_lb_fid, True) 
    print("{} has been saved".format(out_img_fid))
    print("{} has been saved".format(out_lb_fid))

Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192. 148.]
Spacing: (1.1689453125, 1.1689453125, 3.0) -> [1.55859375, 1.55859375, 3.0]
Size (256, 256, 148) -> [192. 192

./processed/image_11.nii.gz has been saved
./processed/label_11.nii.gz has been saved
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Label values: [ 0.  1.  2.  3.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.6497395833333333, 1.6497395833333333, 5.0]
Size (256, 256, 97) -> [192. 192.  97.]
Spacing: (1.2373046875, 1.2373046875, 5.0) -> [1.64973

Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.8092455863952637, 1.8092455863952637, 5.0]
Size (256, 256, 89) -> [192. 192.  89.]
Spacing: (1.3569341897964478, 1.3569341897964478, 5.0) -> [1.809

Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1.572265625, 1.572265625, 3.0) -> [2.0963541666666665, 2.0963541666666665, 3.0]
Size (256, 256, 199) -> [192. 192. 199.]
Spacing: (1

Spacing: (1.425292730331421, 1.425292730331421, 5.0) -> [1.900390307108561, 1.900390307108561, 5.0]
Size (256, 256, 105) -> [192. 192. 105.]
./processed/image_22.nii.gz has been saved
./processed/label_22.nii.gz has been saved
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333, 1.5677083333333333, 5.0]
Size (256, 256, 99) -> [192. 192.  99.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333, 1.5677083333333333, 5.0]
Size (256, 256, 99) -> [192. 192.  99.]
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333, 1.5677083333333333, 5.0]
Size (256, 256, 99) -> [192. 192.  99.]
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333, 1.5677083333333333, 5.0]
Size (256, 256, 99) -> [192. 192.  99.]
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333, 1.5677083333333333, 5.0]
Size (256, 256, 99) -> [192. 192.  99.]
Spacing: (1.17578125, 1.17578125, 5.0) -> [1.5677083333333333

Spacing: (1.23046875, 1.23046875, 5.0) -> [1.640625, 1.640625, 5.0]
Size (256, 256, 100) -> [192. 192. 100.]
./processed/image_26.nii.gz has been saved
./processed/label_26.nii.gz has been saved
Spacing: (1.7089835405349731, 1.7089835405349731, 5.0) -> [2.2786447207132974, 2.2786447207132974, 5.0]
Size (256, 256, 101) -> [192. 192. 101.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.7089835405349731, 1.7089835405349731, 5.0) -> [2.2786447207132974, 2.2786447207132974, 5.0]
Size (256, 256, 101) -> [192. 192. 101.]
Spacing: (1.7089835405349731, 1.7089835405349731, 5.0) -> [2.2786447207132974, 2.2786447207132974, 5.0]
Size (256, 256, 101) -> [192. 192. 101.]
Spacing: (1.7089835405349731, 1.7089835405349731, 5.0) -> [2.2786447207132974, 2.2786447207132974, 5.0]
Size (256, 256, 101) -> [192. 192. 101.]
Spacing: (1.7089835405349731, 1.7089835405349731, 5.0) -> [2.2786447207132974, 2.2786447207132974, 5.0]
Size (256, 256, 101) -> [192. 192. 101.]
Spacing:

Spacing: (1.0390625, 1.0390625, 3.0) -> [1.3854166666666665, 1.3854166666666665, 3.0]
Size (256, 256, 141) -> [192. 192. 141.]
Spacing: (1.0390625, 1.0390625, 3.0) -> [1.3854166666666665, 1.3854166666666665, 3.0]
Size (256, 256, 141) -> [192. 192. 141.]
Spacing: (1.0390625, 1.0390625, 3.0) -> [1.3854166666666665, 1.3854166666666665, 3.0]
Size (256, 256, 141) -> [192. 192. 141.]
Spacing: (1.0390625, 1.0390625, 3.0) -> [1.3854166666666665, 1.3854166666666665, 3.0]
Size (256, 256, 141) -> [192. 192. 141.]
./processed/image_3.nii.gz has been saved
./processed/label_3.nii.gz has been saved
Spacing: (1.5859375, 1.5859375, 3.0) -> [2.114583333333333, 2.114583333333333, 3.0]
Size (256, 256, 118) -> [192. 192. 118.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.5859375, 1.5859375, 3.0) -> [2.114583333333333, 2.114583333333333, 3.0]
Size (256, 256, 118) -> [192. 192. 118.]
Spacing: (1.5859375, 1.5859375, 3.0) -> [2.114583333333333, 2.114583333333333, 3.0]
Si

Spacing: (1.27490234375, 1.27490234375, 3.0) -> [1.6998697916666665, 1.6998697916666665, 3.0]
Size (256, 256, 149) -> [192. 192. 149.]
Spacing: (1.27490234375, 1.27490234375, 3.0) -> [1.6998697916666665, 1.6998697916666665, 3.0]
Size (256, 256, 149) -> [192. 192. 149.]
Spacing: (1.27490234375, 1.27490234375, 3.0) -> [1.6998697916666665, 1.6998697916666665, 3.0]
Size (256, 256, 149) -> [192. 192. 149.]
Spacing: (1.27490234375, 1.27490234375, 3.0) -> [1.6998697916666665, 1.6998697916666665, 3.0]
Size (256, 256, 149) -> [192. 192. 149.]
./processed/image_7.nii.gz has been saved
./processed/label_7.nii.gz has been saved
Spacing: (1.7089835405349731, 1.7089835405349731, 2.5) -> [2.2786447207132974, 2.2786447207132974, 2.5]
Size (256, 256, 150) -> [192. 192. 150.]
Label values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
Spacing: (1.7089835405349731, 1.7089835405349731, 2.5) -> [2.2786447207132974, 2.2786447207132974, 2.5]
Size (256, 256, 150) -> [192. 192. 150.]
Spacing: (1.70